<a href="https://colab.research.google.com/github/alsh02/Mathematical-Explorations/blob/main/%EA%B5%B0%EC%A7%91%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 군집화하기
피파 선수들의 데이터를 바탕으로 200명의 선수들을 상, 중, 하로 군집화해봅시다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from copy import deepcopy

In [ ]:
# 구글 드라이브 마운트하기
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# csv 파일 불러오기
fifa_csv = pd.read_csv("/content/drive/MyDrive/colab_csv/fifa2019.csv")
fifa_df = pd.DataFrame.copy(fifa_csv.sort_values(by='Overall', ascending=False).head(200))
test_features = ['Name', 'Stamina', 'Dribbling', 'ShortPassing', 'Penalties']
test_df = pd.DataFrame(fifa_df, columns=test_features)

In [ ]:
# 선수들 리스트 중 stamina와 dribbling 항목만 가져옴.
all_player_array = np.array(test_df)
players = all_player_array[:, 1:3]

In [ ]:
# 임의의 3개의 중심점 생성
k = 3 # 분류하려는 군집이 상, 중, 하 이므로 3개이다.
centroid_about_stamina = np.random.choice(players[:, 0], k)
centroid_about_dribbling = np.random.choice(players[:, 1], k)
centroids = np.array(list(zip(centroid_about_stamina, centroid_about_dribbling)))

In [ ]:
# 유클리디언 거리를 구하는 함수
def distance(a, b):
  return np.sqrt(np.sum(np.power(a - b, 2)))

In [ ]:
# 중심 좌표를 업데이트하기 위해서 전 상태의 중심 좌표를 저장해놓을 변수 선언
temp_centroids = np.zeros(centroids.shape)

# 모든 데이터의 군집 라벨 저장을 위해 array 선언
clusters = np.zeros(len(players))

# 반복문의 종료 기준 변수 선언. 0일 때 반복문을 종료한다.
stop_flag = distance(centroids, temp_centroids)

In [ ]:
distances = []
while stop_flag != 0: # 중심 좌표가 변하지 않을 때까지 반복
  for i, player in enumerate(players):
    for j, centroid in enumerate(centroids): 
      distances.append(distance(player, centroid)) # 한 데이터와 3개의 중심점까지의 거리 계산
    cluster = np.argmin(distances) # 그 중 가장 작은 거리의 인덱스값을 저장
    clusters[i] = cluster # 그 인덱스값은 군집의 라벨이 됨.
    distances = []
    
  temp_centroids = deepcopy(centroids) # 현재 중심 좌표 array를 복사
  
  # 현재 각각의 군집의 무게중심의 좌표를 구한 뒤 위에서 복사한 중심 좌표와 비교한다.
  for i in range(k):
    points = [players[idx] for idx in range(len(players)) if clusters[idx] == i]
    centroids[i] = np.mean(points)
  stop_flag = distance(centroids, temp_centroids)

In [ ]:
# 1번 군집
plt.scatter(players[clusters == 0,0], players[clusters == 0,1], s=50, c='red', marker='o', edgecolor='black', label='Middle')

# 2번 군집
plt.scatter(players[clusters == 1,0], players[clusters == 1,1], s=50, c='yellow', marker='x', edgecolor='black', label='Low')

# 3번 군집
plt.scatter(players[clusters == 2,0], players[clusters == 2,1], s=50, c='blue', marker='^', edgecolor='black', label='High')

# 군집의 중심 좌표들
plt.scatter(centroids[:, 0], centroids[:, 1], s=250, marker='*', c='black', edgecolor='black', label='Centroids')
plt.legend()
plt.grid()
plt.show()